# Real-Time Intelligence - Manufacturing Operations Solution Installer
This notebook orchestrates the end-to-end deployment of **Real-Time Intelligence - Manufacturing Operations Solution** solution assets into the current Microsoft Fabric workspace using the `fabric-launcher` library.

**Installation tasks**
This notebook performs the following tasks:
1. **🚀 Deployment**: download source code, deploy Fabric items, load reference Data
1. **✅ Post-Deployment Tasks**: Complete post-deployment configuration tasks, such as:
    - Import sample data
    - Move items to appropriate folders

**Expected duration**
This notebook will take about 10 minutes to run. The deployment of Fabric items should take about 5 minutes. An additional 5 minutes will be required to complete the post-deployment tasks, such as generating sample data.

## 🚀 Deployment
Download source code, deploy Fabric items, load reference Data

In [ ]:
%pip install fabric-launcher --quiet
import notebookutils
notebookutils.session.restartPython()

## 2. Configure Settings

Set the GitHub source and the target Lakehouse name.

In [ ]:
from fabric_launcher import GitHubDownloader, LakehouseFileManager

# ── GitHub source settings ────────────────────────────────────────────────────
GITHUB_OWNER     = "slavatrofimov"
GITHUB_REPO      = "udf-fabric-workspace-cicd"
GITHUB_BRANCH    = "main"
GITHUB_UDF_WORKSPACE_PATH = "workspace-udf" # Folder inside the repo where UDF solution is synched
GITHUB_DATA_PATH = "data"                   # Folder inside the repo to download

# ── Fabric bronze Lakehouse target settings ─────────────────────────────────────────
LAKEHOUSE_NAME = "maag_bronze"      # Lakehouse in the current workspace
LAKEHOUSE_TARGET_CSV_FOLDER  = "/"  # Root folder under Lakehouse Files/

# ── Temporary local directory ─────────────────────────────────────────────────
LOCAL_TEMP_DIR = "/tmp/udf_data"    # Where the repo folder is extracted

In [ ]:
from fabric_launcher import FabricLauncher

launcher = FabricLauncher(notebookutils,
    api_root_url = "https://api.fabric.microsoft.com" #Default is https://api.fabric.microsoft.com, but may vary depending on your environment
    )

# Deploy solution with data folders
launcher.download_and_deploy(
    repo_owner=GITHUB_OWNER,
    repo_name=GITHUB_REPO,
    branch = GITHUB_BRANCH,
    workspace_folder=GITHUB_UDF_WORKSPACE_PATH,
    allow_non_empty_workspace=True,
    item_type_stages = [["Lakehouse"],["Notebook","Report","DataAgent","SemanticModel"]],
    validate_after_deployment=True,
    generate_report=False,
    extract_to=LOCAL_TEMP_DIR
)

## ✅ Automated Post-Deployment Tasks

In [ ]:
#Import required libraries
import sempy.fabric as fabric
from fabric_launcher import create_or_update_fabric_item, get_folder_id_by_name, move_item_to_folder, scan_logical_ids

# Initialize Fabric client and workspace
client = fabric.FabricRestClient()
workspace_id = fabric.get_workspace_id()

In [ ]:
file_manager = LakehouseFileManager(notebookutils)

file_manager.copy_folder_to_lakehouse(
    lakehouse_name=LAKEHOUSE_NAME,
    source_folder=LOCAL_TEMP_DIR,
    target_folder=LAKEHOUSE_TARGET_CSV_FOLDER,
    file_patterns=["*.csv"],   # Only upload CSV files
    recursive=True,
)

In [ ]:
# Run a notebook to: Convert and standardize data form Bronze to Silver Lakehouse
result = launcher.run_notebook_synchronous(
    notebook_name="run_bronze_to_silver",
    parameters={},
    timeout_seconds=3600
)

In [ ]:
# Run a notebook to: Segment data from Silver to Gold Lakehouse
result = launcher.run_notebook_synchronous(
    notebook_name="run_silver_to_gold",
    parameters={},
    timeout_seconds=3600
)

In [ ]:
# Move current notebook to the Install folder
move_item_to_folder(
        item_name=notebookutils.runtime.context["currentNotebookName"],
        item_type="Notebook",
        folder_name="notebooks",
        workspace_id=workspace_id,
        client=client
    )
print("✅ Current notebook moved successfully!")

## Next Steps
Your automated solution deployment is complete!

⚠️ Please be sure to refresh your browser window to reflect all newly-deployed items in the Fabric portal!

Review the README document for details on running simulations and exploring your solution.